In [3]:
import pandas as pd
import numpy as np
import matplotlib as plt
from datetime import datetime
%matplotlib inline
train = pd.read_csv("../../../../Kaggle/Red Hat/Data/act_train.csv")
test = pd.read_csv("../../../../Kaggle/Red Hat/Data/act_test.csv")
people = pd.read_csv("../../../../Kaggle/Red Hat/Data/people.csv")

In [4]:
train.head()

,people_id,activity_id,date,activity_category,char_1,char_2,char_3,char_4,char_5,char_6,char_7,char_8,char_9,char_10,outcome
0,ppl_100,act2_1734928,2023-08-26,type 4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,type 76,0
1,ppl_100,act2_2434093,2022-09-27,type 2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,type 1,0
2,ppl_100,act2_3404049,2022-09-27,type 2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,type 1,0
3,ppl_100,act2_3651215,2023-08-04,type 2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,type 1,0
4,ppl_100,act2_4109017,2023-08-26,type 2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,type 1,0


In [5]:
test.head()

,people_id,activity_id,date,activity_category,char_1,char_2,char_3,char_4,char_5,char_6,char_7,char_8,char_9,char_10
0,ppl_100004,act1_249281,2022-07-20,type 1,type 5,type 10,type 5,type 1,type 6,type 1,type 1,type 7,type 4,NaN
1,ppl_100004,act2_230855,2022-07-20,type 5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,type 682
2,ppl_10001,act1_240724,2022-10-14,type 1,type 12,type 1,type 5,type 4,type 6,type 1,type 1,type 13,type 10,NaN
3,ppl_10001,act1_83552,2022-11-27,type 1,type 20,type 10,type 5,type 4,type 6,type 1,type 1,type 5,type 5,NaN
4,ppl_10001,act2_1043301,2022-10-15,type 5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,type 3015


In [6]:
people.head()

,people_id,char_1,group_1,char_2,date,char_3,char_4,char_5,char_6,char_7,...,char_29,char_30,char_31,char_32,char_33,char_34,char_35,char_36,char_37,char_38
0,ppl_100,type 2,group 17304,type 2,2021-06-29,type 5,type 5,type 5,type 3,type 11,...,False,True,True,False,False,True,True,True,False,36
1,ppl_100002,type 2,group 8688,type 3,2021-01-06,type 28,type 9,type 5,type 3,type 11,...,False,True,True,True,True,True,True,True,False,76
2,ppl_100003,type 2,group 33592,type 3,2022-06-10,type 4,type 8,type 5,type 2,type 5,...,False,False,True,True,True,True,False,True,True,99
3,ppl_100004,type 2,group 22593,type 3,2022-07-20,type 40,type 25,type 9,type 4,type 16,...,True,True,True,True,True,True,True,True,True,76
4,ppl_100006,type 2,group 6534,type 3,2022-07-27,type 40,type 25,type 9,type 3,type 8,...,False,False,True,False,False,False,True,True,False,84


In [7]:
print(train.dtypes, "\n")
print(test.dtypes, "\n")
print(people.dtypes, "\n")

people_id            object
activity_id          object
date                 object
activity_category    object
char_1               object
char_2               object
char_3               object
char_4               object
char_5               object
char_6               object
char_7               object
char_8               object
char_9               object
char_10              object
outcome               int64
dtype: object 

people_id            object
activity_id          object
date                 object
activity_category    object
char_1               object
char_2               object
char_3               object
char_4               object
char_5               object
char_6               object
char_7               object
char_8               object
char_9               object
char_10              object
dtype: object 

people_id    object
char_1       object
group_1      object
char_2       object
date         object
char_3       object
char_4       object
char_5       obj

In [8]:
test_ids = test['activity_id']

def preprocess_acts(data, train_set = True):
    
    if train_set:
        data = data.drop(['outcome'], axis = 1)
    data['people_id'] = data['people_id'].apply(lambda x: x.split('_')[1])
    data['people_id'] = pd.to_numeric(data['people_id']).astype(int)
    data = add_date_fields(data)
    data = data.drop(['date','activity_id'], axis = 1)
    columns = list(data.columns)
    
    for col in columns[1:12]:
        data[col] = data[col].fillna('type 0')
        data[col] = data[col].apply(lambda x: x.split(' ')[1])
        data[col] = pd.to_numeric(data[col]).astype(int)
    for col in columns[12:15]:
        data[col] = pd.to_numeric(data[col]).astype(int)
    return data

def preprocess_people(data):
    data = data.drop(['date'], axis=1)
    data['people_id'] = data['people_id'].apply(lambda x: x.split('_')[1])
    data['people_id'] = pd.to_numeric(data['people_id']).astype(int)
    columns = list(data.columns)
    bools = columns[11:]
    strings = columns[1:11]

    for col in bools:
        data[col] = pd.to_numeric(data[col]).astype(int)
    for col in strings:
        data[col] = data[col].fillna('type 0')
        data[col] = data[col].apply(lambda x: x.split(' ')[1])
        data[col] = pd.to_numeric(data[col]).astype(int)
    return data

def add_date_fields(data): 
    data['date_year'] = data['date'].apply(lambda x: str(x.year))
    data['date_month'] = data['date'].apply(lambda x: str(x.month))
    data['date_day'] = data['date'].apply(lambda x: str(x.day))
    #data['date_weekday'] = data['date'].apply(lambda x: str(x.weekday))  
    return data

In [9]:
# convert field 'date' into data type
train['date'] = train['date'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d') )

test['date'] = test['date'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d') )

In [10]:
preprocess_people = preprocess_people(people)
actions_train = preprocess_acts(train)
actions_test = preprocess_acts(test, train_set = False)

In [11]:
# Training data
features = actions_train.merge(preprocess_people, how = 'inner', on = 'people_id')
labels = train['outcome']
#Testing data
test = actions_test.merge(preprocess_people, how = 'inner', on='people_id')

features.sample(10)

,people_id,activity_category,char_1_x,char_2_x,char_3_x,char_4_x,char_5_x,char_6_x,char_7_x,char_8_x,...,char_29,char_30,char_31,char_32,char_33,char_34,char_35,char_36,char_37,char_38
366450,157864,5,0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,79
1545757,357560,5,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
501442,182669,2,0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,96
601053,199970,2,0,0,0,0,0,0,0,0,...,1,0,1,0,1,1,1,0,0,94
557080,192138,4,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,1,1,68
1055746,279007,4,0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,97
2074199,78246,5,0,0,0,0,0,0,0,0,...,0,1,1,0,1,1,0,1,1,37
1181679,294918,3,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
698334,217765,5,0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,41
691793,216413,2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,78


In [12]:
#Split training set
from sklearn.cross_validation import train_test_split


In [19]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.cross_validation import cross_val_score

clf = RandomForestClassifier(n_estimators = 12, max_depth = 35, n_jobs = -1, warm_start = True )
clf.fit(features, labels)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=35, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=12, n_jobs=-1,
            oob_score=False, random_state=None, verbose=0, warm_start=True)

In [15]:
probability = clf.predict_proba(features)
predictions = probability[:,1]
scores = cross_val_score(estimator = clf, X=features, y=labels, cv = 10, scoring = 'roc_auc')
print("ROC AUC: %0.2f (+/- 0.2f)" %(scores.mean(),scores.std()))


Area under ROC 0.9999982564199579


In [20]:
test_probability = clf.predict_proba(test)
test_predictions = test_probability[:,1]
output = pd.DataFrame({'activity_id': test_ids, 'outcome':test_predictions})
output.head()

,activity_id,outcome
0,act1_249281,0.583333
1,act2_230855,0.416667
2,act1_240724,0.666667
3,act1_83552,0.833333
4,act2_1043301,0.916667


In [22]:
output.to_csv('submission2.csv',index = False)